# Imports

In [1]:
from pathlib import Path
import torch
from torch import nn
from torch import optim
from mltrainer import metrics, Trainer, TrainerSettings, ReportTypes
from mltrainer.preprocessors import BasePreprocessor
import mlflow
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope
from mads_datasets import DatasetFactoryProvider, DatasetType
from loguru import logger
from datetime import datetime

# Datastreamer

In [2]:
datafactory = DatasetFactoryProvider.create_factory(DatasetType.FASHION)
batchsize = 64
preprocessor = BasePreprocessor()
streamers = datafactory.create_datastreamer(batchsize=batchsize, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]
train_streamer = train.stream()
valid_streamer = valid.stream()

2023-12-16 17:19:36.664 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/fashionmnist
2023-12-16 17:19:36.666 | INFO     | mads_datasets.base:download_data:124 - File already exists at /home/azureuser/.cache/mads_datasets/fashionmnist/fashionmnist.pt


# Model

In [3]:
# Although this program will likely only run on the vm,
# and the vm lacks gpu support, a gpu check is added in case the
# program is run outside of the vm.

device = "gpu" if torch.cuda.is_available() else "cpu"
print(f"Model training takes place on the {device}")

# Model definement
class CNN(nn.Module):
    def __init__(self, filters: int, units1: int, units2: int, dropout: float=0.2, input_size: tuple=(32, 1, 28, 28)):
        super().__init__()

        self.convolutional_layers = nn.Sequential(
            nn.Conv2d(1, filters, kernel_size=3),
            nn.BatchNorm2d(filters),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filters, filters, kernel_size=3),
            nn.BatchNorm2d(filters),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(filters, filters, kernel_size=3),
            nn.BatchNorm2d(filters),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

        # In order to solve the potential problem of connecting the image layers
        # with the linear layers, an AvgPool2d is added based on the size of the
        # activationmap from the convolutional layers.
        # Resulting in (batch, activationmaps, 1, 1) before flattening.
        activaton_map_size = self.conv_test(input_size)
        self.aggregated_layer = nn.AvgPool2d(activaton_map_size)

        self.dense_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(filters, units1),
            nn.BatchNorm1d(units1),
            nn.ReLU(),
            nn.Linear(units1, units2),
            nn.BatchNorm1d(units2),
            nn.ReLU(),
            nn.Dropout1d(dropout),
            nn.Linear(units2, 10)
        )

    def conv_test(self, input_size):
        input_size_matrix = torch.ones(input_size)
        conv_layers_output = self.convolutional_layers(input_size_matrix)
        return conv_layers_output.shape[-2:]
    
    def forward(self, input):
        conv_layers_output = self.convolutional_layers(input)
        agg_layer_output = self.aggregated_layer(conv_layers_output)
        dense_layers_output = self.dense_layers(agg_layer_output)
        return dense_layers_output

model = CNN(filters=32, units1=64, units2=32, dropout=0.2).to(device)

Model training takes place on the cpu


# Optimizer, loss & accuracy

In [4]:
optimizer = optim.Adam
loss_function = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()

# MLflow

In [9]:
experiment_path = "hyper_parameter_exercise"
mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment(experiment_path)

<Experiment: artifact_location='/home/azureuser/code/ADS-DeepLearning-project/hyper_parameter_exercise/notebook/mlruns/1', creation_time=1702738838363, experiment_id='1', last_update_time=1702738838363, lifecycle_stage='active', name='hyper_parameter_exercise', tags={}>

# Hyperopt

In [6]:
model_directory = Path("../../models/mnist").resolve()
if not model_directory.exists():
    model_directory.mkdir()
    print(f"Created {model_directory}")

Created /home/azureuser/code/ADS-DeepLearning-project/models/mnist


In [18]:
trainer_settings = TrainerSettings(
    epochs=3,
    metrics=[accuracy],
    logdir="modellog",
    train_steps=100,
    valid_steps=100,
    reporttypes=[ReportTypes.MLFLOW]
)

def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "convolutional_network")
        mlflow.set_tag("dev", "tom")
        mlflow.log_params(params)
        mlflow.log_param("batchsize", f"{batchsize}")

        model = CNN(**params)
        trainer = Trainer(
            model=model,
            settings=trainer_settings,
            loss_fn=loss_function,
            optimizer=optimizer,
            traindataloader=train_streamer,
            validdataloader=valid_streamer,
            scheduler=optim.lr_scheduler.ReduceLROnPlateau
        )
        trainer.loop()

        tag = datetime.now().strftime("%Y%m%d-%H%M")
        model_path = model_directory / (tag + "model.pt")
        torch.save(model, model_path)

        mlflow.log_artifact(local_path=model_path, artifact_path="pytorch_models")
        
        return {"loss": trainer.test_loss, "status": STATUS_OK}

In [10]:
search_space = {
    "filters": scope.int(hp.quniform("filters", 16, 128, 8)),
    "units1": scope.int(hp.quniform("units1", 32, 128, 8)),
    "units2": scope.int(hp.quniform("units2", 32, 128, 8)),
}

In [19]:
best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=3,
    trials=Trials()
)

  0%|          | 0/3 [00:00<?, ?trial/s, best loss=?]

2023-12-16 16:38:11.675 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to modellog/20231216-163811
2023-12-16 16:38:11.677 | INFO     | mltrainer.trainer:__init__:65 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|##########| 100/100 [00:08<00:00, 11.42it/s]
2023-12-16 16:38:24.378 | INFO     | mltrainer.trainer:report:171 - Epoch 0 train 1.1408 test 0.6503 metric ['0.7848']
100%|##########| 100/100 [00:08<00:00, 11.21it/s]
2023-12-16 16:38:37.159 | INFO     | mltrainer.trainer:report:171 - Epoch 1 train 0.8795 test 0.5477 metric ['0.8103']
100%|##########| 100/100 [00:08<00:00, 11.37it/s]
2023-12-16 16:38:49.885 | INFO     | mltrainer.trainer:report:171 - Epoch 2 train 0.8491 test 0.5622 metric ['0.7984']
2023-12-16 16:38:49.886 | INFO     | mltrainer.trainer:__call__:214 - best loss: 0.5477, current loss 0.5622.Counter 1/10.
100%|##########| 3/3 [00:38<00:00, 12.74s/it]


 33%|███▎      | 1/3 [00:38<01:16, 38.38s/trial, best loss: 0.5621517878770829]

2023-12-16 16:38:50.028 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to modellog/20231216-163850
2023-12-16 16:38:50.030 | INFO     | mltrainer.trainer:__init__:65 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|##########| 100/100 [00:07<00:00, 12.62it/s]
2023-12-16 16:39:01.555 | INFO     | mltrainer.trainer:report:171 - Epoch 0 train 1.2881 test 0.8327 metric ['0.8000']
100%|##########| 100/100 [00:07<00:00, 12.67it/s]
2023-12-16 16:39:13.115 | INFO     | mltrainer.trainer:report:171 - Epoch 1 train 0.9599 test 0.6290 metric ['0.8066']
100%|##########| 100/100 [00:07<00:00, 12.63it/s]
2023-12-16 16:39:24.665 | INFO     | mltrainer.trainer:report:171 - Epoch 2 train 0.8804 test 0.5891 metric ['0.8059']
100%|##########| 3/3 [00:34<00:00, 11.55s/it]


 67%|██████▋   | 2/3 [01:13<00:36, 36.26s/trial, best loss: 0.5621517878770829]

2023-12-16 16:39:24.815 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to modellog/20231216-163924
2023-12-16 16:39:24.817 | INFO     | mltrainer.trainer:__init__:65 - Found earlystop_kwargs in settings.Set to None if you dont want earlystopping.
100%|##########| 100/100 [00:12<00:00,  8.19it/s]
2023-12-16 16:39:42.099 | INFO     | mltrainer.trainer:report:171 - Epoch 0 train 1.1468 test 0.7034 metric ['0.7700']
100%|##########| 100/100 [00:12<00:00,  8.32it/s]
2023-12-16 16:39:59.102 | INFO     | mltrainer.trainer:report:171 - Epoch 1 train 0.9015 test 0.5816 metric ['0.8097']
100%|##########| 100/100 [00:12<00:00,  8.31it/s]
2023-12-16 16:40:16.138 | INFO     | mltrainer.trainer:report:171 - Epoch 2 train 0.8679 test 0.5117 metric ['0.8327']
100%|##########| 3/3 [00:51<00:00, 17.11s/it]


100%|██████████| 3/3 [02:04<00:00, 41.54s/trial, best loss: 0.5116935002803803]


best_result